In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Transaction Configuration

Shows transaction configuration created in LUSID by default, and how to create customised transaction types in transaction configuration. 

Attributes
----------
transaction configuration
transactions
output transactions
holdings"""

toggle_code("Toggle Docstring")

# Transaction Configuration


## Table of contents
1. [Overview](#1.-Overview)
2. [Setup](#2.-Setup)
3. [Load Data](#3.-Load-Data)
   * [3.1 Create Portfolio](#3.1-Create-Portfolio)
   * [3.2 Create Instruments](#3.2-Create-Instruments)
   * [3.3 Insert Transactions](#3.3-Insert-Transactions)
4. [Examine default Transaction Configuration and Resultant Holdings](#4.-Examine-default-Transaction-Configuration-and-Resultant-Holdings)
   * [4.1 Transaction Type 'Buy'](#4.1-Transaction-Type-Buy)
   * [4.2 Transaction Type 'FundsIn'](#4.2-Transaction-Type-FundsIn)
   * [4.3 Resultant Holdings](#4.3-Resultant-Holdings)
5. [Use Customised Transaction Configuration to Represent Movements](#5.-Use-Customised-Transaction-Configuration-to-Represent-Movements)
    * [5.1 Set Up Property Definition 'Transaction/TradeWithCashReserve/CashReserveLuid'](#5.1-Set-Up-Property-Definition-'Transaction/TradeWithCashReserve/CashReserveLuid')
    * [5.2 Set Up Side Definition 'CashReserve'](#5.2-Set-Up-Side-Definition-'CashReserve')
    * [5.3 Set Up Transaction Type 'BuyFromCashReserve'](#5.3-Set-Up-Transaction-Type-'BuyFromCashReserve')
    * [5.4 Insert 'Buy Amazon' Transaction with Transaction Type 'BuyFromCashReserve'](#5.4-Insert-'Buy-Amazon'-Transaction-with-Transaction-Type-'BuyFromCashReserve')
    * [5.5 Examine Holdings after Update Transaction with 'BuyFromCashReserve' Transaction Type](#5.5-Examine-Holdings-after-Update-Transaction-with-'BuyFromCashReserve'-Transaction-Type)
6. [Data Cleanup](#6.-Data-Cleanup)

## 1. Overview

[Transaction Configuration](https://support.lusid.com/knowledgebase/article/KA-01872) is a core feature of LUSID which provides a lot of flexibility in setting transaction types and their underlying movements. In addition to transaction types and configrations provided by default, other entity fields and properties can also be utilised when creating your own transaction configuration. 

This notebook demonstrates how these fields and properties can be used to customise Transaction Configuration to suit your needs.

## 2. Setup

In [2]:
# Import general purpose packages
import os
import json
from datetime import datetime
from datetime import timedelta
import pytz

# Import lusid specific packages
import lusid
import lusid.models as models
from lusid.exceptions import ApiException
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.seed_sample_data import seed_data
from lusidtools.cocoon.utilities import create_scope_id

# Import data wrangling packages
import pandas as pd

pd.set_option("display.max_columns", None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

Load a mapping file for DataFrame headers for handling responses from 
+ `build transaction`
+ `get holdings`
+ `list_instruments`

In [3]:
with open(r"config/build_transactions_mapping.json") as mappings_file:
    build_transactions_json_mapping = json.load(mappings_file)

with open(r"config/get_holdings_mapping.json") as mappings_file:
    get_holdings_json_mapping = json.load(mappings_file)

with open(r"config/list_instruments_mapping.json") as mappings_file:
    list_instruments_json_mapping = json.load(mappings_file)

Create new scope, portfolio code, and dates to be used throughout the notebook

In [4]:
scope="notebook-txn-config"
portfolio_code="EQUITY" + "-" + create_scope_id()
transaction_date=datetime(year=2020, month=10, day=1, tzinfo=pytz.UTC)
settlement_date=datetime(year=2020, month=10, day=5, tzinfo=pytz.UTC)

Define portfolios/transaction portfolios/instruments/transaction configuration/property definitions API, which will be used in later steps.

In [5]:
portfolios_api = api_factory.build(lusid.api.PortfoliosApi)
transaction_portfolios_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
transaction_configuration_api = api_factory.build(lusid.api.TransactionConfigurationApi)
system_configuration_api = api_factory.build(lusid.api.SystemConfigurationApi)
property_definitions_api = api_factory.build(lusid.api.PropertyDefinitionsApi)

Keep a copy of the original transaction types, side definitions, and transaction configuration itself, in order to restore them after the notebook is completed. 

In [6]:
original_transaction_types=system_configuration_api.list_configuration_transaction_types()
original_side_definitions=original_transaction_types.side_definitions
original_transaction_configs=original_transaction_types.transaction_configs

## 3. Load Data

### 3.1 Create Portfolio

Create a transaction portfolio to insert transactions in later steps.

In [7]:
# Create a transaction portfolio, with base currency GBP

try:
    create_transaction_portfolio_response=transaction_portfolios_api.create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=models.CreateTransactionPortfolioRequest(
            code=portfolio_code,
            display_name=portfolio_code,
            base_currency='GBP',
            created='2000-01-01T00:00:00+00:00'
        )
    )
    display(f"Portfolio {portfolio_code} in scope {scope} created.")

except ApiException as e: 
    if json.loads(e.body)["name"] == "PortfolioWithIdAlreadyExists":
        display(f"Portfolio {portfolio_code} in scope {scope} already exists.")
    

'Portfolio EQUITY-3baa-d8d5-55ac-98 in scope notebook-txn-config created.'

### 3.2 Create Instruments

Create 3 instruments for later use: 

- VOD.L equity 
- AMZN equity
- USD Cash Reserve as SimpleInstrument

In [8]:
upsert_instruments_response = instruments_api.upsert_instruments(
    scope=scope,
    request_body=
    {
        'create-VOD.L':
        models.InstrumentDefinition(
            name='Vodafone PLC',
            identifiers={
                'Sedol': models.InstrumentIdValue('BH4HKS3'),
                'Isin': models.InstrumentIdValue('GB00BH4HKS39'),
                'ClientInternal': models.InstrumentIdValue('cid_VOD.L')
            },
            definition=models.Equity(
                instrument_type="Equity",
                dom_ccy="GBP",
            )
        ),
        'create-AMZN':
        models.InstrumentDefinition(
            name='Amazon',
            identifiers={
                'Figi': models.InstrumentIdValue('BBG000BVPV84'),
                'ClientInternal': models.InstrumentIdValue('cid_AMZN')
            },
            definition=models.Equity(
                instrument_type="Equity",
                dom_ccy="USD",
            )
        ),
        'create-cash-reserve-usd':
        models.InstrumentDefinition(
            name='Cash Reserve (USD)',
            identifiers={
                'ClientInternal': models.InstrumentIdValue('cid_cash_reserve_usd')
            },
            definition=models.SimpleInstrument(
                asset_class='Money',
                dom_ccy='USD',
                simple_instrument_type='None',
                instrument_type='SimpleInstrument'
            )
        )
    }
)

# In addition, we will keep record of LusidInstrumentId for that USD Cash Reserve for later use.

usd_cash_reserve_luid = upsert_instruments_response.values['create-cash-reserve-usd'].identifiers['LusidInstrumentId']

### 3.3 Insert Transactions

Insert 3 transactions to transaction portfolio: 

1. Put 150,000.00 GBP of funds into Portfolio
2. Buy 250 VOD.L equity stocks, at price 1.00 GBP for each stock
3. Put 150,000.00 USD of funds as USD Cash Reserve into Portfolio

In [9]:
upsert_ccy_gbp_transaction_response=transaction_portfolios_api.upsert_transactions(
    scope=scope,
    code=portfolio_code,
    transaction_request=[
        models.TransactionRequest(
            transaction_id='trd-0001',
            type='FundsIn',
            instrument_identifiers={
                'Instrument/default/Currency':'GBP'
            },
            transaction_date=transaction_date.isoformat(),
            settlement_date=settlement_date.isoformat(),
            units=150000,
            transaction_price=models.TransactionPrice(
                price=1,
                type='Price'
            ),
            total_consideration=models.CurrencyAndAmount(
                amount=150000,
                currency='GBP'
            )
        )
    ]
)

In [10]:
upsert_vod_transaction_response=transaction_portfolios_api.upsert_transactions(
    scope=scope,
    code=portfolio_code,
    transaction_request=[
        models.TransactionRequest(
            transaction_id='trd-0002',
            type='Buy',
            instrument_identifiers=
            {
                'Instrument/default/Isin': 'GB00BH4HKS39'
            },
            transaction_date=transaction_date.isoformat(),
            settlement_date=settlement_date.isoformat(),
            units=250,
            transaction_price=models.TransactionPrice(
                price=1,
                type='Price'
            ),
            total_consideration=models.CurrencyAndAmount(
                amount=250,
                currency='GBP'
            )
        )
    ]
)

In [11]:
upsert_usd_cash_reserve_response=transaction_portfolios_api.upsert_transactions(
    scope=scope,
    code=portfolio_code,
    transaction_request=[
        models.TransactionRequest(
            transaction_id='trd-0003',
            type='FundsIn',
            instrument_identifiers={
                'Instrument/default/ClientInternal':'cid_cash_reserve_usd'
            },
            transaction_date=transaction_date.isoformat(),
            settlement_date=settlement_date.isoformat(),
            units=100000,
            transaction_price=models.TransactionPrice(
                price=1,
                type='Price'
            ),
            total_consideration=models.CurrencyAndAmount(
                amount=100000,
                currency='USD'
            )
        )
    ]
)

## 4. Examine default Transaction Configuration and Resultant Holdings

Let's look at `movements` of associated with two `transaction types` which we used above - "Buy" and "FundsIn". 

### 4.1 Transaction Type `Buy`

"Buy" type indicates stock side (Side1) "moves up" and currency (Side2) "moves down" when buying a stock. This is one of the most commonly used transaction types 

Both `Side1` and `Side2` are default [side definitions](https://support.lusid.com/knowledgebase/article/KA-01875/en-us) in LUSID.

In [12]:
get_buy_transaction_configuration_response = transaction_configuration_api.get_transaction_type(
    source='default',
    type='Buy'
)

lusid_response_to_data_frame(get_buy_transaction_configuration_response.movements)

,movement_types,side,direction,properties,mappings,movement_options
0,StockMovement,Side1,1,{},[],[]
1,CashCommitment,Side2,-1,{},[],[]


### 4.2 Transaction Type `FundsIn`

'FundsIn' states only 'funds' go 'into' the portfolio.

In [13]:
get_funds_in_transaction_configuration_response = transaction_configuration_api.get_transaction_type(
    source='default',
    type='FundsIn'
)

lusid_response_to_data_frame(get_funds_in_transaction_configuration_response.movements)

,movement_types,side,direction,properties,mappings,movement_options
0,CashReceivable,Side1,1,{},[],[]


### 4.3 Resultant Holdings

When getting holdings for portfolio, you should see 5 holdings: 

1. 250 units of VOD.L @ 1.00 GBP, with total cost of 250.00 GBP
2. 150,000 'units' of USD Cash Reserve
3. 149,750.00 GBP in portfolio, due to
  - 150000.00 GBP as "funds" got "in"
  - 250.00 GBP used to buy 250 VOD.L stocks

In [14]:
get_initial_holdings_response = transaction_portfolios_api.get_holdings(
    scope=scope,
    code=portfolio_code,
    property_keys=["Instrument/default/Name"]
)

lusid_response_to_data_frame(
    get_initial_holdings_response,
    column_name_mapping=get_holdings_json_mapping,
    rename_properties=True)

,instrument_scope,LusidInstrumentId,SubHoldingKeys,InstrumentName,SourcePortfolioId,SourcePortfolioScope(default-Properties),HoldingType,Units,SettledUnits,Amount-Cost,Currency-Cost,Amount-CostPortfolioCcy,Currenct-CostPortfolioCcy,currency,holding_type_name
0,default,LUID_00003D6P,{},Vodafone PLC,EQUITY-3baa-d8d5-55ac-98,notebook-txn-config,P,250.0,250.0,250.0,GBP,250.0,GBP,GBP,Position
1,default,CCY_GBP,{},GBP,EQUITY-3baa-d8d5-55ac-98,notebook-txn-config,B,149750.0,149750.0,149750.0,GBP,149750.0,GBP,GBP,Balance
2,default,LUID_00003D6Q,{},Cash Reserve (USD),EQUITY-3baa-d8d5-55ac-98,notebook-txn-config,P,100000.0,100000.0,100000.0,USD,0.0,GBP,USD,Position


## Use Customised Transaction Configuration to Represent Movements

This section demonstrates setting up customised Transaction Configuration to change underlying movements of a transaction type. Specificially, we will reference properties on a transaction to define side definitions.

In this example, we will update transaction configuration with a new transaction type, so that the funds to buy 300 AMZN stocks will be drawn from USD Cash Reserve instead of USD Currency.

### 5.1 Set Up Property Definition 'Transaction/TradeWithCashReserve/CashReserveLuid'

First, create a new property definition with property key 'Transaction/TradeWithCashReserve/DeductFromLuid' for later use. 

This property will be stamped on the transaction if the funds of buying an instrument needs to be deducted from a specific cash reserve.

In [15]:
cash_reserve_luid_prop_scope='TradeWithCashReserve'
cash_reserve_luid_prop_code='CashReserveLuid'

try: 
    create_cash_reserve_luid_property_definition_response = property_definitions_api.create_property_definition(
        create_property_definition_request=models.CreatePropertyDefinitionRequest(
            domain='Transaction',
            scope=cash_reserve_luid_prop_scope,
            code=cash_reserve_luid_prop_code,
            display_name='Cash Reserve LUID',
            data_type_id=models.ResourceId('system','string'),
            life_time='Perpetual',
            constraint_style='Property',
            property_description='Cash Reserve LUID',
        )
    )
    display(f"Property Definition 'Transaction/{cash_reserve_luid_prop_scope}/{cash_reserve_luid_prop_code}' created.")
    
except ApiException as e: 
    if json.loads(e.body)["name"] == "PropertyAlreadyExists":
        display(f"Property Definition 'Transaction/{cash_reserve_luid_prop_scope}/{cash_reserve_luid_prop_code}' already exists.")    

"Property Definition 'Transaction/TradeWithCashReserve/CashReserveLuid' created."

### 5.2 Set Up Side Definition 'CashReserve'

Next, create a new [`side definition`](https://support.lusid.com/knowledgebase/article/KA-01875/) to define cash reserve as a side in any underlying movement.

This side definition references security to use 'Transaction/TradeWithCashReserve/CashReserveLuid' property to define the LusidInstrumentId (LUID) of the Cash Reserve.

In [16]:
cash_reserve_side_str='CashReserve'

cash_reserve_side_def_response = transaction_configuration_api.set_side_definition(
    side=cash_reserve_side_str,
    side_definition_request=models.SideDefinitionRequest(
        security='Transaction/TradeWithCashReserve/CashReserveLuid',
        currency='Txn:TradeCurrency',
        rate='Txn:TradeToPortfolioRate',
        units='Txn:TotalConsideration',
        amount='Txn:TotalConsideration'
    )
)

### 5.3 Set Up Transaction Type 'BuyFromCashReserve'

Lastly, create a new transaction type "BuyFromCashReserve" in "default" source. 

When this transaction type is used, it states to deduct from cash reserve stated in 'Transaction/TradeWithCashReserve/DeductFromLuid' property in a transaction.

In [17]:
buy_from_cash_reserve_txn_type_str='BuyFromCashReserve'

buy_from_cash_reserve_in_txn_type_response = transaction_configuration_api.set_transaction_type(
    source='default',
    type=buy_from_cash_reserve_txn_type_str,
    transaction_type_request=models.TransactionTypeRequest(
        aliases=[
            models.TransactionTypeAlias(
                type=buy_from_cash_reserve_txn_type_str,
                description='Buy from Cash Reserve',
                transaction_class='Basic',
                transaction_roles='LongLonger'
            )
        ],
        movements=[
            models.TransactionTypeMovement(
                movement_types='StockMovement',
                side='Side1',
                direction=1
            ),
            models.TransactionTypeMovement(
                movement_types='CashCommitment',
                side=cash_reserve_side_str,
                direction=-1
            )
        ]
    )
)

### 5.4 Insert 'Buy Amazon' Transaction with Transaction Type 'BuyFromCashReserve'

Now, we insert a transaction for buying Amazon stocks. 

There are two main points to note:

1. This transaction has an additional property key 'Transaction/TradeWithCashReserve/CashReserve'. The value of this property is the LusidInstrumentId (LUID) of the same USD Cash Reserve we created earlier.
2. Transaction Type is 'BuyFromCashReserve', which was set up previously. By using this transaction type, the funds used to buy these stock will come from the instrument with LusidInstrumentId stated in 'Transaction/TradeWithCashReserve/CashReserve', which is the USD Cash Reserve created in previous steps.

In [18]:
upsert_amzn_transaction_response=transaction_portfolios_api.upsert_transactions(
    scope=scope,
    code=portfolio_code,
    transaction_request=[
        models.TransactionRequest(
            transaction_id='trd-0004',
            type=buy_from_cash_reserve_txn_type_str,
            instrument_identifiers={
                'Instrument/default/ClientInternal':'cid_AMZN'
            },
            transaction_date=transaction_date.isoformat(),
            settlement_date=settlement_date.isoformat(),
            units=300,
            transaction_price=models.TransactionPrice(
                price=95,
                type='Price'
            ),
            total_consideration=models.CurrencyAndAmount(
                amount=28500,
                currency='USD'
            ),
            properties={
                'Transaction/TradeWithCashReserve/CashReserveLuid':
                models.PerpetualProperty(
                    key='Transaction/TradeWithCashReserve/CashReserveLuid',
                    value=models.PropertyValue(
                        label_value=usd_cash_reserve_luid
                    )
                )
            }
        )
    ]
)

### 5.5 Examine Holdings after Update Transaction with 'BuyFromCashReserve' Transaction Type

Now, get holdings for the portfolio again.

Take note of the holdings associated with the 'Cash Reserve (USD)' instrument. This is because the "BuyFromCashReserve" transaction type has used the funds in USD Cash Reserve to buy those 300 Amazon stock, which costs 28,500 USD in total.

This leaves 100,000 - 28,500 = 71,500 in USD Cash Reserve.

In [19]:
get_holdings_response = transaction_portfolios_api.get_holdings(
    scope=scope,
    code=portfolio_code,
    property_keys=["Instrument/default/Name"]
)

lusid_response_to_data_frame(
    get_holdings_response,
    column_name_mapping=get_holdings_json_mapping,
    rename_properties=True)

,instrument_scope,LusidInstrumentId,SubHoldingKeys,InstrumentName,SourcePortfolioId,SourcePortfolioScope(default-Properties),HoldingType,Units,SettledUnits,Amount-Cost,Currency-Cost,Amount-CostPortfolioCcy,Currenct-CostPortfolioCcy,currency,holding_type_name
0,default,LUID_00003D6P,{},Vodafone PLC,EQUITY-3baa-d8d5-55ac-98,notebook-txn-config,P,250.0,250.0,250.0,GBP,250.0,GBP,GBP,Position
1,default,LUID_00003D6R,{},Amazon,EQUITY-3baa-d8d5-55ac-98,notebook-txn-config,P,300.0,300.0,28500.0,USD,0.0,GBP,USD,Position
2,default,CCY_GBP,{},GBP,EQUITY-3baa-d8d5-55ac-98,notebook-txn-config,B,149750.0,149750.0,149750.0,GBP,149750.0,GBP,GBP,Balance
3,default,LUID_00003D6Q,{},Cash Reserve (USD),EQUITY-3baa-d8d5-55ac-98,notebook-txn-config,P,71500.0,71500.0,71500.0,USD,0.0,GBP,USD,Position


## 6. Data Cleanup

Delete portfolio used in this notebook

In [20]:
try:
    portfolios_api.delete_portfolio(
        scope=scope,
        code=portfolio_code)
    display(f"Portfolio {portfolio_code} in scope {scope} deleted.")
except ApiException as e: 
    if json.loads(e.body)["name"] == "PortfolioNotFound":
        display(f"Portfolio {portfolio_code} in scope {scope} already deleted.")

'Portfolio EQUITY-3baa-d8d5-55ac-98 in scope notebook-txn-config deleted.'

Delete the transaction type created in this notebook

In [21]:
try:
    transaction_configuration_api.delete_transaction_type(
        source='default',
        type=buy_from_cash_reserve_txn_type_str
    )
    display(f"Transaction Configuration Type '{buy_from_cash_reserve_txn_type_str}' from source 'default' deleted.")
except:
    display(f"Transaction Configuration Type '{buy_from_cash_reserve_txn_type_str}' from source 'default' already deleted.")

"Transaction Configuration Type 'BuyFromCashReserve' from source 'default' deleted."

Delete the property definition created in this notebook

In [22]:
try: 
    property_definitions_api.delete_property_definition(
        domain='Transaction',
        scope=cash_reserve_luid_prop_scope,
        code=cash_reserve_luid_prop_code
    )
    display(f"Property Definition 'Transaction/{cash_reserve_luid_prop_scope}/{cash_reserve_luid_prop_code}' deleted.")
except ApiException as e: 
    if json.loads(e.body)["name"] == "PropertyNotDefined":
        display(f"Property Definition 'Transaction/{cash_reserve_luid_prop_scope}/{cash_reserve_luid_prop_code}' does not exist.")

"Property Definition 'Transaction/TradeWithCashReserve/CashReserveLuid' deleted."

Reset transaction types in this notebook

In [23]:
reset_transaction_types=system_configuration_api.set_configuration_transaction_types(
    transaction_set_configuration_data_request=models.TransactionSetConfigurationDataRequest(
        transaction_config_requests=original_transaction_configs,
        side_config_requests=original_side_definitions
    )
)